In [ ]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [ ]:
rho = 11e3
Omega = 2*np.pi/86400
U0 = 8.7e-5
R = 3486e3

In [ ]:
def fx(alpha, lat, NoverOmega):
    expr = (NoverOmega**2 - 1)*(1 - 4*np.sin(lat)**2) + 4*np.cos(lat)**2*np.sin(alpha)**2
    
    if expr < 0:
        return 0.0

    return -np.sin(lat)*np.sqrt(expr)*np.cos(alpha)**2

In [ ]:
def Fxintalpha(NoverOmega,lat):
 return quad(fx, 0, 2*np.pi, args=(lat, NoverOmega), limit=1000, epsabs=1e-10, epsrel=1e-10)[0]

In [ ]:
def fx_trad(alpha, lat, NoverOmega):
    expr = (NoverOmega**2 - 1)*(1 - 4*np.sin(lat)**2)
    
    if expr < 0:
        return 0.0

    return -np.sin(lat)*np.sqrt(expr)*np.cos(alpha)**2

In [ ]:
def Fxintalpha_trad(NoverOmega,lat):
 return quad(fx_trad, 0, 2*np.pi, args=(lat, NoverOmega), limit=1000, epsabs=1e-10, epsrel=1e-10)[0]

In [ ]:
def fy(alpha, lat, NoverOmega):
    expr = (NoverOmega**2 - 1)*(1 - 4*np.sin(lat)**2) + 4*np.cos(lat)**2*np.sin(alpha)**2
    
    if expr < 0:
        return 0.0

    return np.sqrt(expr)*np.sin(alpha)**2

In [ ]:
def Fyintalpha(NoverOmega,lat):
 return quad(fy, 0, 2*np.pi, args=(lat, NoverOmega), limit=1000, epsabs=1e-10, epsrel=1e-10)[0]

In [ ]:
def fy_trad(alpha, lat, NoverOmega):
    expr = (NoverOmega**2 - 1)*(1 - 4*np.sin(lat)**2)
    
    if expr < 0:
        return 0.0

    return np.sqrt(expr)*np.sin(alpha)**2

In [ ]:
def Fyintalpha_trad(NoverOmega,lat):
 return quad(fy_trad, 0, 2*np.pi, args=(lat, NoverOmega), limit=1000, epsabs=1e-10, epsrel=1e-10)[0]

In [ ]:
# Vectorize over latitude
latitudes = np.linspace(0, np.pi/2, 100) 
Fx_vec = np.vectorize(lambda lat: Fxintalpha(0, lat))
form_drag_x = Fx_vec(latitudes)

Fx_vec_trad = np.vectorize(lambda lat: Fxintalpha_trad(0, lat))
form_drag_x_trad = Fx_vec_trad(latitudes)

In [ ]:
# Vectorize over latitude
latitudes = np.linspace(0, np.pi/2, 100) 
Fy_vec = np.vectorize(lambda lat: Fyintalpha(0, lat))
form_drag_y = Fy_vec(latitudes)

Fy_vec_trad = np.vectorize(lambda lat: Fyintalpha_trad(0, lat))
form_drag_y_trad = Fy_vec_trad(latitudes)

In [ ]:
# Plotting
plt.rc('text', usetex=True)
plt.plot(np.degrees(latitudes), form_drag_x, label="zonal")
plt.plot(np.degrees(latitudes), form_drag_y, label="meridional")
plt.xlabel("Latitude (degrees)")
plt.title("Geometric factor")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Wrapper for latitude integration, including Jacobian (R^2 cos(lat)) and accounting for the integral over longitudes
def integrand(lat, NoverOmega):
    return R * np.pi * (Fyintalpha(NoverOmega,lat)-np.sin(lat)*Fxintalpha(NoverOmega, lat)) * R**2 * np.cos(lat)

In [ ]:
N_values = np.linspace(0., 5, 41) 
results = []

for NoverOmega in N_values:
    torque, _ = quad(
        integrand,
        -np.pi/2,
        np.pi/2,
        args=(NoverOmega),
        limit=1000,
        epsabs=1e-10,
        epsrel=1e-10
    )
    print(torque)
    results.append(torque)

In [ ]:
# Wrapper for latitude integration, including Jacobian (R^2 cos(lat))
def integrand_trad(lat, NoverOmega):
    return R * np.pi * (Fyintalpha_trad(NoverOmega,lat)-np.sin(lat)*Fxintalpha_trad(NoverOmega, lat)) * R**2 * np.cos(lat)

In [ ]:
N_values = np.linspace(0., 5, 41) 
results_trad = []

for NoverOmega in N_values:
    torque, _ = quad(
        integrand_trad,
        -np.pi/2,
        np.pi/2,
        args=(NoverOmega),
        limit=1000,
        epsabs=1e-10,
        epsrel=1e-10
    )
    print(torque)
    results_trad.append(torque)

In [ ]:
k0 = 2*np.pi/1e6
H = 0.7
hrms = 5.67e3

In [ ]:
As = 5.8531e34
Af = 9.0583e36
A  = 8.0115e37
Am = A - (As+Af)
wf = 2.4e-11

In [ ]:
plt.rc('text', usetex=True)

# Secondary y-axis
sf = Af * wf * Omega
ax = plt.gca()
ax.secondary_yaxis('right', functions=(lambda y: y / sf, lambda y: y * sf))\
         .set_ylabel(r'$\mathrm{Re}(K_\mathrm{cmb})$', fontsize=14)

plt.plot(N_values,(rho/4/np.pi**3)*(hrms**2*k0*H/(2*H-1))*(U0*Omega)*np.array(results), marker='o', linestyle='-', label="exact")
plt.plot(N_values,(rho/4/np.pi**3)*(hrms**2*k0*H/(2*H-1))*(U0*Omega)*np.array(results_trad), marker='o', linestyle='-', label="traditional approximation")
plt.xlabel(r'$N/\Omega$', fontsize=14)
plt.ylabel(r'Torque (Nm)', fontsize=14)
plt.legend()
plt.grid(True)

plt.savefig("Torque.pdf")
plt.show()